# 01.02-maj-extract-DAG-features

Extract values that are relevant to DAG.

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def run_query(db_path, query):
    """Query a database and return a pandas DataFrame."""
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

## High School

Relevant features
- GPA
- Habits of Mind
- Actions
- SAT scores

In [3]:
hs_path = "../data/00-external/HIGH_SCHOOL.db"

hs_query = """
SELECT [SubjectI.D.] as SUBJID,
    SATMATH,
    SATWriting,
    SATVerbal,
    CASE [Whatwasyouraveragegradeinhighschool?]
        WHEN "A or A+" THEN 4.0
        WHEN "A-" THEN 3.7
        WHEN "B+" THEN 3.3
        WHEN "B" THEN 3.0
        WHEN "B-" THEN 2.7
        WHEN "C+" THEN 2.3
        WHEN "C" THEN 2.0
        WHEN "D" THEN 1.0
    END AS GPA,
    CASE [HabitsofMind:Askquestionsinclass]
        WHEN "Frequently" THEN 2
        WHEN "Occasionally" THEN 1
        WHEN "Not at all" THEN 0
    END AS hab1,
    CASE [HabitsofMind:Supportyouropinionswithalogicalargument]
        WHEN "Frequently" THEN 2
        WHEN "Occasionally" THEN 1
        WHEN "Not at all" THEN 0
    END AS hab2,
    CASE [HabitsofMind:Seeksolutionstoproblemsandexplainthemtoothers]
        WHEN "Frequently" THEN 2
        WHEN "Occasionally" THEN 1
        WHEN "Not at all" THEN 0
    END as hab3,
    CASE [HabitsofMind:Reviseyourpaperstoimproveyourwriting]
        WHEN "Frequently" THEN 2
        WHEN "Occasionally" THEN 1
        WHEN "Not at all" THEN 0
    END as hab4
    FROM 'HIGH SCHOOL'
    WHERE SATMath IS NOT NULL
        AND SATWriting IS NOT NULL
        AND SATVerbal IS NOT NULL
        AND GPA IS NOT NULL
        AND hab1 IS NOT NULL
        AND hab2 IS NOT NULL
        AND hab3 IS NOT NULL
        AND hab4 IS NOT NULL
        AND Surveyyear == 2010;
"""
hs = run_query(hs_path, hs_query)
hs.dropna(inplace=True)

In [4]:
hs

,SUBJID,SATMath,SATWriting,SATVerbal,GPA,hab1,hab2,hab3,hab4
0,884230,450,420,430,3.3,1,1,2,2
1,884232,430,430,470,2.7,2,2,1,1
2,884233,430,320,320,2.0,1,2,0,1
3,884247,560,540,580,4.0,2,2,2,2
4,884266,440,400,410,3.0,2,2,1,1
...,...,...,...,...,...,...,...,...,...
99691,960942,230,238,240,3.7,2,2,2,2
99692,960943,500,600,500,3.0,2,2,2,0
99693,960945,580,700,600,3.3,1,1,1,2
99694,960946,570,570,670,3.0,1,2,2,1


## Demographics

In [5]:
demo_path = "../data/00-external/DEMOGRAPHICS.db"

query_demographics = """
SELECT [SubjectI.D.] as SUBJID,
    State AS state,
    [Yoursex:] AS gender,
    [Race/EthnicityGroup] AS race,
    ParentalIncome AS income
FROM DEMOGRAPHICS
WHERE Surveyyear == 2010
    AND state IS NOT NULL
    AND gender != 0
    AND race IS NOT NULL
    AND income IS NOT NULL;
"""

demo_df = run_query(demo_path, query_demographics)
demo_df.dropna(inplace=True)

# TFS

In [6]:
tfs_path = "../data/00-external/TFS_CHOICE_2008_2010.db"

    #    CASE WHEN ACCPT1ST = 2 THEN 1 ELSE 0 END as ACCPT1ST
query_tfs = """
SELECT SUBJID,
    CASE CHOICE
        WHEN 5 THEN -1
        WHEN 4 THEN -2
        WHEN 2 THEN -3
        WHEN 1 THEN -4
    END AS choice,
    CASE ACCPT1ST
        WHEN 1 THEN 0
        WHEN 2 THEN 1
    END AS accepted,
    ACERECODE as school,
    NUMAPPLY AS numapply
FROM CHOICE
WHERE Year == 2010
    AND CHOICE IS NOT NULL
    AND accepted IS NOT NULL
    AND school IS NOT NULL
    AND numapply IS NOT NULL;
"""

tfs= run_query(tfs_path, query_tfs)
tfs['choice'] = np.abs(tfs['choice'])
tfs.dropna(inplace=True)

In [7]:
tfs.dropna()

,SUBJID,choice,accepted,school,numapply
0,884186,2,1,2192,4
1,884187,2,1,2192,3
2,884188,4,1,2192,10
3,884189,3,1,2192,4
4,884190,2,0,2192,4
...,...,...,...,...,...
254363,960945,1,1,3494,5
254364,960946,1,1,3494,4
254365,960947,1,1,3494,2
254366,960948,1,1,3494,6


## Merging

In [8]:
print(hs.shape)
print(demo_df.shape)
print(tfs.shape)

(99696, 9)
(223016, 5)
(254368, 5)


In [9]:
merged_df = pd.merge(hs, demo_df, how='left', on="SUBJID")
print(merged_df.shape)
merged_df.isna().sum()

(99696, 13)


SUBJID           0
SATMath          0
SATWriting       0
SATVerbal        0
GPA              0
hab1             0
hab2             0
hab3             0
hab4             0
state         6898
gender        6898
race          6898
income        6898
dtype: int64

There were ~7k observations in the high school data frame that weren't in demographics

In [10]:
merged_df.dropna(inplace=True)
merged_df.shape

(92798, 13)

In [11]:
merged_df = pd.merge(merged_df, tfs, how='left', on='SUBJID')

In [12]:
merged_df.dropna(inplace=True)
print(merged_df.shape)

(91264, 17)


In [13]:
merged_df.to_csv("../data/01-raw/dag.csv", index=False)

In [14]:
merged_df

,SUBJID,SATMath,SATWriting,SATVerbal,GPA,hab1,hab2,hab3,hab4,state,gender,race,income,choice,accepted,school,numapply
0,884230,450,420,430,3.3,1,1,2,2,AL,Female,Black,"$25,000-$29,999",4.0,1.0,2192.0,9
1,884232,430,430,470,2.7,2,2,1,1,AL,Male,Black,"$150,000-$199,999",1.0,1.0,2192.0,6
2,884233,430,320,320,2.0,1,2,0,1,AL,Male,Black,"$40,000-$49,999",4.0,0.0,2192.0,4
3,884247,560,540,580,4.0,2,2,2,2,AL,Female,Black,"$100,000-$149,999",1.0,1.0,2192.0,9
4,884307,520,570,510,3.3,2,1,2,2,AL,Male,Black,"$10,000-$14,999",4.0,1.0,2192.0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92793,886635,300,500,400,3.7,1,1,1,2,FL,Female,Two or more race/ethnicity,"$250,000 or more",1.0,1.0,1691.0,6
92794,886640,670,660,650,3.7,1,2,2,2,FL,Female,White,"$75,000-$99,999'",1.0,1.0,1691.0,3
92795,886642,610,780,740,4.0,1,2,2,1,FL,Female,Two or more race/ethnicity,"$60,000-$74,999",1.0,1.0,1691.0,9
92796,886648,580,510,550,3.3,2,2,2,1,FL,Male,White,"$60,000-$74,999",2.0,1.0,1691.0,6
